In [1]:
import urllib
import urllib.request
import http
import http.cookiejar
import cx_Oracle
import time
import js2py
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
spider='wapjs_esxvm2208'
spider_db_user='NTTASK'
spider_db_pass='NTTASK'
spider_db='ESXVM0504'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1024x768")
browser = webdriver.Chrome(options=chrome_options)
browser.get("http://wapjs.189.cn/tysh/pages/kdSpeedUp/speedUpIndexNew.html")

In [ ]:
def get_kdinfo(En_CertID):
    url = 'http://wapjs.189.cn/zt-business/broadBandBuiness/selectUserBroadAccNbr'
    opener = urllib.request.build_opener()
    #添加headers
    headers = [('Referer','http://wapjs.189.cn/tysh/pages/kdSpeedUp/speedUpIndexNew.html?ztInterSource=300177&platform=wap&callback=252133'),\
               ('Accept-Language','zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
               ('Cache-Control','max-age=0'),\
               ('Content-Type','application/x-www-form-urlencoded'),\
               ('Origin',url),\
               ('Connection','keep-alive'),\
               ('User-Agent','Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1'),\
               ('Accept','application/json, text/javascript, */*; q=0.01')]
    opener.addheaders = headers

    #添加cookie
    cookie_filename = 'cookie_189_cn.txt'
    #cookies = http.cookiejar.CookieJar()
    cookies = http.cookiejar.MozillaCookieJar(cookie_filename)
    cookies.load(ignore_discard=True, ignore_expires=True)
    change_cookie = http.cookiejar.Cookie(
        version=0, name="cityCode", value="js_nt",
        port=None, port_specified=False,
        domain="js.189.cn",
        domain_specified=True, domain_initial_dot=False,
        path="/", path_specified=True, secure=False,
        expires=None, discard=True,
        comment=None, comment_url=None,
        rest={"HttpOnly": None}, rfc2109=False)
    #cookies.set_cookie(change_cookie)
    cookie_handler = urllib.request.HTTPCookieProcessor(cookies)
    opener.add_handler(cookie_handler)
    #添加代理
    proxy_handler = urllib.request.ProxyHandler({'https':'221.178.146.113:61388','http':'221.178.146.113:61388'})
    #opener.add_handler(proxy_handler)

    #添加PostData
    #data = {"accNbr":"32062519621126691X","areaCode":"0513","isIdCard":"1"}
    #Plain_PostData = "accountType=2;idCard={};areaCode=0513;actionCode=jsztActionCode_mallbroadBandBuinessselectUserBroadAccNbr;PathCode=mall-00003;channelCode_common=300177;pubAreaCode=025;pushUserId=jszt_912507;coachUser=;userLogAccNbrType=;userLogAccNbr=;userTokenAccNbrType=2;ztVersion=4.0;ztInterSource=300177;pubToken=undefined;".format(CertID)
    #En_CertID = get_encode(Plain_PostData)
    data = {"para":En_CertID}
    post_data = urllib.parse.urlencode(data).encode(encoding='UTF8')
    #print(post_data)
    try:
        request = opener.open(url,post_data)
        kd_info=request.read().decode(encoding='utf8',errors='ignore')
    except Exception as ex:
        print(ex)
        kd_info='Exception!'
    return kd_info

def get_js():
    f = open("app.rsa.min.js",'r',encoding='UTF-8')
    line = f.readline()
    htmlstr = ''
    while line:
        htmlstr = htmlstr + line
        line = f.readline()
    return htmlstr

def get_encode(plain_str):
    context = js2py.EvalJs()
    context.execute(get_js())
    print(plain_str)
    Encode_str = context.ztEncrptyUtil.encode(plain_str)
    return Encode_str

In [ ]:
while True:
    dbconn=cx_Oracle.connect(spider_db_user,spider_db_pass,spider_db)
    dbcur=dbconn.cursor()
    smt_sql = "UPDATE kd_dx_customer a SET a.spider_source = '{}' WHERE a.certid like '3206%' AND a.spider_source IS NULL AND ROWNUM <= 2".format(spider)
    dbcur.execute(smt_sql)
    dbconn.commit()
    smt_sql = "SELECT a.certid FROM kd_dx_customer a WHERE a.spider_source = '{}' AND a.kd_info IS NULL".format(spider)
    dbcur.execute(smt_sql)
    dbrlt=dbcur.fetchall()
    for CertInfo in dbrlt:
        print(CertInfo[0])
        Plain_PostData="accountType=2;idCard={};areaCode=0513;actionCode=jsztActionCode_mallbroadBandBuinessselectUserBroadAccNbr;PathCode=mall-00003;channelCode_common=300177;pubAreaCode=025;pushUserId=jszt_912507;coachUser=;userLogAccNbrType=;userLogAccNbr=;userTokenAccNbrType=2;ztVersion=4.0;ztInterSource=300177;pubToken=undefined;".format(CertInfo[0])
        print(Plain_PostData)
        time.sleep(30)
        En_CertID = browser.execute_script('return ztEncrptyUtil.encode("{}")'.format(Plain_PostData))
        print(En_CertID)
        DX_kd_info=get_kdinfo(En_CertID)        
        print(DX_kd_info)
        smt_sql = "update kd_dx_customer a SET a.kd_info='{}',a.check_time=SYSDATE WHERE a.certid = '{}' AND a.spider_source = '{}'".format(DX_kd_info,CertInfo[0],spider)
        dbcur.execute(smt_sql)
        dbconn.commit()
    dbcur.close()
    dbconn.close()
#print(dbrlt)

In [ ]:
#request = opener.open(url1)
#request_read = request.read()
#print(request_read.decode(encoding='UTF8',errors='ignore'))


In [ ]:
request = opener.open(url1,post_data)
a=request.read()
print(a.decode(encoding='utf8',errors='ignore'))

In [ ]:
#登录成功后，保存cookies
cookies.save(ignore_discard=True, ignore_expires=True)

In [ ]:
print(a)
a.decode(encoding='UTF8')

In [ ]:
get_kdinfo('32062119641004832X')

In [ ]:
dbrlt[1][0]

In [ ]:
get_kdinfo('320601197701019413')

In [3]:
print(browser.page_source)

<html class="ui-mobile"><head><base href="http://wapjs.189.cn/tysh/pages/kdSpeedUp/speedUpIndexNew.html">
	<meta name="description" content="">
	<meta name="viewport" content="width=device-width, height=device-height, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no">
	<meta content="initial-scale=1.0,user-scalable=no,maximum-scale=1" media="(device-height: 568px)" name="viewport"> 
	<meta content="yes" name="apple-mobile-web-app-capable"> 
	<meta content="black" name="apple-mobile-web-app-status-bar-style"> 
	<meta content="telephone=no" name="format-detection">
	<meta name="viewport" content="width=device-width, height=device-height">
	<meta http-equiv="content-type" content="text/html; charset=UTF-8">
	<meta http-equiv="pragma" content="no-cache">
	<meta name="format-detection" content="telephone=no">
	<script type="text/javascript" src="../../resources/js/load.js"></script><link rel="stylesheet" type="text/css" href="../../resources/css/jquery.mobile-1.4.5.

In [5]:
browser.execute_script('return i.param')

JavascriptException: Message: javascript error: i is not defined
  (Session info: headless chrome=74.0.3729.169)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Linux 4.15.0-50-generic x86_64)
